In [ ]:
import os
os.environ['https_proxy']='127.0.0.1:17890'
os.environ['http_proxy']='127.0.0.1:17890'
from datasets import load_dataset,Dataset,concatenate_datasets
import pandas as pd
import sys,os
import datasets
from tqdm import tqdm


In [ ]:
from datasets import load_dataset,load_from_disk
ds_0=load_from_disk("/data02/users/lz/code/UICoder/datasets/chunk[08-14]-arrow")
ds_1=load_from_disk("/data02/starmage/datasets/cc/arrows_5-7")

In [ ]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("openai-community/gpt2")


In [ ]:
tokenizer(["af,fdfad", "fdffaaa4564654"], max_length=4096, truncation=True)

In [ ]:
import re

def split_html_css(text):
    html_content = re.sub(f'\s+', ' ', text)

    # 正则表达式匹配<style>标签及其内容
    style_tag_pattern = r'<style[^>]*>([\s\S]*?)<\/style>'

    # 使用re.DOTALL使得.可以匹配包括换行符在内的任意字符
    # re.IGNORECASE忽略大小写
    style_content = re.search(style_tag_pattern, html_content, re.DOTALL | re.IGNORECASE)

    # 提取<style>标签内的CSS内容
    if style_content:
        css_content = style_content.group(1).strip()
    else:
        css_content = ''

    # 移除HTML中的<style>标签及其内容
    clean_html = re.sub(style_tag_pattern, '', html_content, flags=re.DOTALL | re.IGNORECASE)

    return  [css_content,clean_html]
    

In [ ]:
step = 512*3
start_index = 0
for i, j in enumerate(tqdm(range(0, len(ds_3), step))):
    ds_3.select(range(j, min(j+step, len(ds_3)))).to_parquet(f"/data02/starmage/datasets/cc/all-parquets/{(start_index+i):05}.parquet",batch_size=step)

In [ ]:
import importlib
import scripts.data_cc_pipeline.post_process as post_process
importlib.reload(post_process)
from scripts.data_cc_pipeline.post_process import detect_lang
[i[0]["label"] for i in detect_lang(["abc", "bcd"], "cuda")]

In [ ]:
from pathlib import Path
from datasets import concatenate_datasets
from tqdm import tqdm

all_parquets = []

for root, dirs, files in os.walk("/data02/users/lz/code/UICoder/datasets/c4-wash/scored-chunks/chunk15-format-scored-parquet"):
    root = Path(root)
    for file in files:
        if file.endswith('.parquet'):
            all_parquets.append(root/file)
            
ds_items = []
for file in tqdm(all_parquets):
    try:
        df_item = pd.read_parquet(file)
        ds_item = Dataset.from_pandas(df_item)
        score = int(file.name.split('.')[0])
        print(score)
        #pos= str(file).find('chunk')
        #chunk = int(str(file)[pos+5:pos+7])
        ds_item = ds_item.add_column("score", [score]*len(ds_item))
        ds_items.append(ds_item)
    except Exception as e:
        print(f"file to read {file}, {e}")
                
ds = concatenate_datasets(ds_items)

    

In [ ]:
ds.save_to_disk("/data02/starmage/datasets/cc/arrows_15_processed_1")

In [ ]:
ds_1=ds.cast_column("image", datasets.Image(decode=True))
ds_2 = concatenate_datasets([ds_0,ds_1])

In [ ]:
ds.save_to_disk("/data02/starmage/datasets/cc/arrows_5-7")

In [ ]:
p1="/data03/starmage/projects/UICoder/outputs/tmp111"
ds1=ds_0.select(range(0,10000))
ds1.save_to_disk(p1)

In [ ]:
ds1[0]

In [ ]:
ds1[0]

In [ ]:
ds1

In [ ]:
from scripts.data_cc_pipeline.post_process import detect_lang

def func(items): 
    try:
        items["scale"]=[item.size for item in items['image']]    
        items["lang"]=[i[0]["label"] for i in detect_lang(items["text"], device="cuda:7")]
        contents = [  split_html_css(text) for text in items["text"]]
        items["tokens"] = [ list(map(len,tokenizer(con,  max_length=10240, truncation=True)["input_ids"]))  for con in contents]        
        return items    
    except Exception as e:
        print(e)
        return None
ds_tmp = ds_tmp.map(func, num_proc=32, batched=True, batch_size=128)
ds_tmp.save_to_disk(p1)

In [ ]:
ds_tmp[0]

In [ ]:
aaa = [[11,1,2],[3,4]]
list(map(len, aaa))

In [ ]:
ds=load_from_disk("/data02/starmage/datasets/cc/arrows_5-7_processed")

In [ ]:
d2 = Dataset.from_parquet("/data02/users/lz/code/UICoder/datasets/c4-wash/scored-chunks/chunk15-format-scored-parquet/2.parquet")

In [ ]:
ds.select(range(0,100)).save_to_disk("/data03/starmage/projects/UICoder/outputs/tmp123")

In [ ]:
ds1= datasets.load_from_disk("/data02/starmage/datasets/cc/arrows_15_processed")

In [ ]:
import base64
import hashlib
from PIL import Image
from io import BytesIO

def image_to_hash(image_pil: Image):
    # Convert PIL Image to Bytes
    byte_arr = BytesIO()
    image_pil.save(byte_arr, format='PNG')
    # Perform Base64 encoding
    hash_object = hashlib.sha256(byte_arr.getvalue())  
    return hash_object.hexdigest()

In [ ]:
def dudup(items, ids):
    hash_values = [image_to_hash(im) for im in items["image"]] 
    return {"id":ids, "hash":hash_values}


ds1 = ds1.map(dudup, num_proc=64, batched=True, batch_size=256, with_indices=True  )
df = ds1.select_columns(['id', 'hash']).to_pandas()
no_dup_idx =  df[~df.duplicated('hash')]["id"].to_list()
print(f"duplicated count {len(ds1)-len(no_dup_idx)}")
ds_dedup=ds1.select(no_dup_idx)

In [ ]:
ds_dedup=ds_dedup.select_columns( ['image', 'bbox', 'text', 'score', 'scale', 'lang', 'tokens', 'hash'])

In [ ]:
ds_dedup.save_to_disk("/data02/starmage/datasets/cc/arrows_15_processed_nodup", num_proc=32)

In [ ]:
ds0= datasets.load_from_disk("/data02/starmage/datasets/cc/arrows_15_processed_nodup")
ds1= datasets.load_from_disk("/data02/starmage/datasets/cc/arrows_5-7_processed_nodup")
df0=ds0.select_columns(["score", "tokens", "scale", "lang", 'hash']).to_pandas()
df1=ds1.select_columns(["score", "tokens", "scale", "lang", 'hash']).to_pandas()
ds=concatenate_datasets([ds0, ds1])
df = ds.select_columns(["score", "tokens", "scale", "lang", 'hash']).to_pandas()
#df = df[~df.duplicated('hash')]

In [ ]:
df[df.duplicated('hash')].iloc[0].index

In [ ]:
import random
id = random.choice(range(len(ds1)))
ds1[id]['hash'] == df.iloc[id+len(ds0)]["hash"]

In [ ]:
import random
id = random.choice(range(len(df1)))

df1.loc[id]['hash']== image_to_hash(ds_train[id]['image'])

In [ ]:
hashes1 = pd.read_csv("/data02/starmage/datasets/cc/8-15_hash.csv")

In [ ]:
hashes1.duplicated('hash').sum()

In [ ]:
def get_nodup_ids(df_to):   
    df3= pd.concat([df_to["hash"], hashes1])
    ids_nodup= df_to[~df3.duplicated('hash', keep='last')[:len(df_to)]].index
    print(len(df_to)-len(ids_nodup))
    return ids_nodup

In [ ]:
ids_nodup_0 = get_nodup_ids(df0)
ds0_nodup = ds0.select(ids_nodup_0)
ids_nodup_1 = get_nodup_ids(df1)
ds1_nodup = ds1.select(ids_nodup_1)
ds_to_sample = concatenate_datasets([ds0_nodup, ds1_nodup])

In [ ]:

def sample_bench(ds_to_sample:datasets.Dataset, n =256*7):
    df_to_sample = ds_to_sample.select_columns(['score', 'scale', 'lang', 'tokens', 'hash']).to_pandas()
    df_to_sample["len1"]=df_to_sample["tokens"].apply(lambda x : x[0]+x[1])
    def map_values(row):
        if row <= 2048:
            return 'short'
        elif 2048 < row <= 4096:
            return 'mid'
        else:
            return 'long'
    df_to_sample["split"] = df_to_sample["len1"].apply(map_values)
    df_to_sample=df_to_sample[df_to_sample['score'] >=4]

    def sample_bench(df_all, m, values=["split"]):    
        df_sampled = df_all.groupby(values, group_keys=False).apply(lambda x: x.sample(min(m, len(x))))
        return df_sampled

    df_samples = sample_bench(df_to_sample, n)
    ids = df_samples.index
    ds_sampled = ds_to_sample.select(ids)
    ds_sampled = ds_sampled.add_column('split', df_samples['split'].to_list())
    ds_odd = ds_to_sample.select(list(set(range(len(ds_to_sample)))-set(ids)))
    return ds_sampled, ds_odd


In [ ]:
ds_sampled, ds_odd = sample_bench(ds_to_sample)

In [ ]:
ds_odd.save_to_disk("/data02/starmage/datasets/cc/train_arrows_5-8_15", num_proc= 16)

In [ ]:
import random
df_all = ds_sampled.select_columns(['split']).to_pandas()
id_short = list(df_all[df_all['split']=='short'].index)
id_mid = list(df_all[df_all['split']=='mid'].index)
id_long = list(df_all[df_all['split']=='long'].index)

columns = ['image', 'bbox', 'text', 'scale', 'lang', 'tokens', 'hash']
ds_short = ds_sampled.select(id_short).select_columns(columns)
ds_mid = ds_sampled.select(id_mid).select_columns(columns)
ds_long = ds_sampled.select(id_long).select_columns(columns)

In [ ]:
len(id_short),len(id_mid),len(ds_long)

In [ ]:
import clip
import torch
device='cuda:7'
CLIP_MODEL, CLIP_PREPROCESS = clip.load("ViT-B/32", device=device)
def encode_imgs(ims,device=device):
    with torch.no_grad():
        img_tmps = torch.stack([CLIP_PREPROCESS(im) for im in ims]).to(device)
        img_feas = CLIP_MODEL.encode_image(img_tmps).cpu()
    return img_feas   
def img_sim(im1_h, im2_h):
    return torch.nn.functional.cosine_similarity(im1_h, im2_h, dim=0).item()

In [ ]:
from torch.utils.data import DataLoader
def dedup_bench(ds, save_path):
    all_clips=[]
    def col_fn(samples):
        items=[s["image"] for s in samples]
        return items
    dl = DataLoader(ds, batch_size=32, collate_fn=col_fn, num_workers=1, shuffle=False)
    for batch in tqdm(dl):
        feats=encode_imgs(batch)
        all_clips.extend(list(feats))
    df_clip = pd.DataFrame({'clip':all_clips})
    max_sims=[[],[]]
    for i in tqdm(range(len(df_clip))):
        max_sim=-1
        max_j=None
        for j in range(len(df_clip)):
            if j==i:
                continue

            sim = img_sim(df_clip.iloc[i]['clip'], df_clip.iloc[j]['clip'])
            if sim > max_sim:            
                max_sim = sim
                max_j = j
        max_sims[0].append(max_sim)
        max_sims[1].append(max_j)
    df_sims= pd.DataFrame({'sim':max_sims[0], 'id':max_sims[1]})
    df_sims_less=df_sims[df_sims.sim<0.95].sort_values(['sim'])
    df_select = df_sims_less.iloc[:min(256+60, len(df_sims_less))]
    ds_nodup = ds.select(df_select.index)
    ds_nodup.to_parquet(save_path)
    return ds_nodup

In [ ]:
len(ds_short)

In [ ]:
short_good = dedup_bench(ds_short, '/data02/starmage/datasets/cc/bench/short.parquet')

In [ ]:
mid_good = dedup_bench(ds_mid, '/data02/starmage/datasets/cc/bench/mid.parquet')
long_good = dedup_bench(ds_long, '/data02/starmage/datasets/cc/bench/long.parquet')

In [ ]:
ds11= datasets.load_from_disk('/data02/starmage/datasets/cc/train_arrows_5-8_15')



In [ ]:
ds11[0]['tokens']

In [ ]:
ds_short.to_parquet("/data02/starmage/datasets/cc/bench_256+40/short.parquet")
ds_mid.to_parquet("/data02/starmage/datasets/cc/bench_256+40/mid.parquet")
ds_long.to_parquet("/data02/starmage/datasets/cc/bench_256+40/long.parquet")

In [ ]:
ds_sampled.to_pandas().duplicated('hash').sum()

In [ ]:
ds_train_0 = datasets.load_from_disk("/data02/starmage/datasets/cc/arrows_8-14_processed").select_columns(['score', 'scale', 'lang', 'tokens'])
ds_train_1 = datasets.load_from_disk("/data02/starmage/datasets/cc/train_arrows_5-8_15").select_columns(['score', 'scale', 'lang', 'tokens'])

In [ ]:
ds_train_0 = datasets.load_from_disk("/data02/starmage/datasets/cc/arrows_8-14_processed")
ds_train_1 = datasets.load_from_disk("/data02/starmage/datasets/cc/train_arrows_5-8_15")
hashes_train_0 = pd.read_csv("/data02/starmage/datasets/cc/8-14_hash.csv")
no_dup_ids = hashes_train_0[~hashes_train_0.duplicated('hash')].index
ds_train_0_nodup = ds_train_0.select(no_dup_ids)

In [ ]:
len(ds_total)

In [ ]:
ds_total = concatenate_datasets([ds_train_1, ds_train_0_nodup])

In [ ]:
from git import Repo
import time
from tqdm import tqdm
import traceback

os.environ['https_proxy']='127.0.0.1:17890'
os.environ['http_proxy']='127.0.0.1:17890'
repo_path="/data03/starmage/datasets/cc/final"
repo = Repo(repo_path)
def git_push(repo:Repo, file_path, msg):
    try:
        repo.git.add(file_path)
        repo.git.commit('-m', msg)
        repo.git.push()
        return True
    except Exception as e:
        print('Push failed:', e)
        print(traceback.format_exc())
        return False
chunk_size = 1536
start_chunk = 0
for id,start in tqdm(enumerate(range(chunk_size*start_chunk, len(ds_total), chunk_size))):
    fpath=f"{repo_path}/data/{id+start_chunk:05d}.parquet"
    ds_total.select(range(start,min(start+chunk_size, len(ds_total)))).to_parquet(fpath)
    #while not git_push(repo, fpath, f"commit chunk {id+start_chunk}."):
        #time.sleep(5)
    

In [ ]:
len(ds_total)

In [ ]:
repo_path="/data02/starmage/datasets/cc/tmp"
repo = Repo(repo_path)
def is_file_committed(repo, file_path):
    committed = False    
    loginfo = repo.git.log('--', file_path)
    print(loginfo)
    if loginfo:  # if loginfo is not empty, the file has been committed
        committed = True
    return committed

In [ ]:
is_file_committed(repo, "/data02/starmage/datasets/cc/tmp/data/0002.parquet")

In [ ]:
for k in  ds_total[0].keys():
    print(k, type(ds_total[0][k]))

In [ ]:
df_stat=pd.concat([ds_train_0_nodup.to_pandas(), ds_train_1.to_pandas()])

In [ ]:
from PIL import Image
im = Image.open('/data03/starmage/projects/UICoder/data/bad/0.png')
type(im)

In [ ]:
ds_tmp = datasets.Dataset.from_parquet('/data02/starmage/datasets/cc/bench_256+40/short.parquet')
im1 = ds_tmp[154]['image']
im2 = ds_tmp[172]['image']
im1.size,im2.size

In [ ]:
print(sys.getrecursionlimit())

In [ ]:
feas = encode_imgs([ds_tmp[0]['image'],ds_tmp[1]['image']])
feas[0].shape


In [ ]:
ds_clip = datasets.Dataset.from_dict({'clip':feas})

In [ ]:
with open('/data03/starmage/projects/UICoder/outputs/eval_vision2ui-design2code_20240515022540/efbda2a06816ea51c22928270aaece6e/prediction.html', 'r') as f:
    str1 = f.read()
len(str1)

In [17]:
csv_dirs=["/data03/starmage/projects/UICoder/outputs/eval_gpt4v",
          "/data03/starmage/projects/UICoder/outputs/eval_cogvlm",         
         "/data03/starmage/projects/UICoder/outputs/eval_ws",
         #"/data03/starmage/projects/UICoder/outputs/eval_stage2",
         "/data03/starmage/projects/UICoder/outputs/eval_d2c",
         "/data03/starmage/projects/UICoder/outputs/eval_stage0",
         "/data03/starmage/projects/UICoder/outputs/eval_stag0ws",
         ]

In [ ]:
import os,sys
import pandas as pd


def format_decimal(num):
    formatted = "{:.2f}".format(num)
    if formatted.startswith("0"):
        return formatted[1:]
    else:
        return formatted


def format_csv(path):
    with open(path) as f:
        data= f.read()
    splits = data.split('\n')
    if len(splits) != 2:
        df_tmp=pd.read_csv(path)
        df_tmp.columns = [c.strip() for c in df_tmp.columns]
        return df_tmp
    head, tail=data.split('\n')
    
    with open('tmp.csv', 'w') as f:
        f.write(f"{head}\n")
    key = '/data03'
    start = 0
    while start<len(tail):
        pos1 = tail.find(key, start)
        if pos1==-1:
            break
        pos2 = tail.find(key, pos1+1)
        if pos2==-1:
            break
        pos3=tail.find(key, pos2+1)
        line = tail[pos1:pos3]
        start = pos3
        with open('tmp.csv', 'a+') as f:
            f.write(f"{line}\n")
    df_tmp = pd.read_csv('tmp.csv')
    df_tmp.columns = [c.strip() for c in df_tmp.columns]
    return df_tmp

rows=[]
column_names=[]
for dir in csv_dirs:
    for root,dirs,files in os.walk(dir):
        if len(files) > 0 and files[0] != 'metrics_result.csv':
            continue
        files = sorted(files, key=lambda x : x.split('_')[0])
        for file in files:
            if file == 'metrics_result.csv':
                row=[]
                model_name = root.split('/')[-2].split('_')[-1]
                dataset_name = 'v2u_'+root.split('/')[-1].split('_')[-2]
                full_path = os.path.join(root, file)
                #df = pd.read_csv(full_path)
                print(full_path)
                df=format_csv(full_path)
                df['visual_score']=df[['block_match', 'text_match', 'position_match', 'text_color_match', 'clip_score']].mean(axis=1)
                row.extend([model_name, dataset_name])
                columns = []
                for c in df.columns:
                    if c not in ["origin","pred"]:
                        columns.append(c)
                        value=f"{df[c].mean():.2f} (±{df[c].std():.2f})"
                        # $0.01_{\pm.03}$
                        #s1 = format_decimal(df[c].std())                
                        #value=f'${df[c].mean():.2f}_{{±{s1}}}$'
                        row.append(value)                           
                rows.append(row) 
                if len(column_names) ==0:
                    column_names = ['model', 'dataset']+columns
                
                

In [18]:
df_res = pd.DataFrame(rows, columns=column_names)
sort_keys = ['dataset', 'tree_rouge_1','block_match', 'position_match', 'text_match', 'text_color_match', 'clip_score']
df1 = df_res.sort_values(sort_keys)
df1.to_csv('res.csv')
df1

,model,dataset,bleu,rouge,tree_bleu,tree_rouge_1,mse_value,ssim_value,clip_sim,block_match,text_match,position_match,text_color_match,clip_score,visual_score
3,cogvlm,v2u_long,0.22 (±0.28),0.26 (±0.28),0.04 (±0.11),0.01 (±0.01),0.36 (±0.30),0.60 (±0.13),0.65 (±0.10),0.22 (±0.32),0.43 (±0.45),0.33 (±0.36),0.32 (±0.38),0.66 (±0.26),0.39 (±0.30)
6,ws,v2u_long,0.11 (±0.16),0.13 (±0.17),0.21 (±0.22),0.03 (±0.03),0.39 (±0.27),0.61 (±0.15),0.64 (±0.11),0.13 (±0.22),0.59 (±0.37),0.50 (±0.33),0.54 (±0.36),0.65 (±0.30),0.48 (±0.27)
16,stag0ws,v2u_long,0.15 (±0.17),0.17 (±0.17),0.15 (±0.15),0.04 (±0.04),0.44 (±0.33),0.59 (±0.15),0.65 (±0.11),0.14 (±0.23),0.47 (±0.42),0.40 (±0.36),0.32 (±0.32),0.55 (±0.35),0.38 (±0.29)
9,d2c,v2u_long,0.33 (±0.29),0.41 (±0.26),0.43 (±0.23),0.06 (±0.03),0.37 (±0.28),0.61 (±0.11),0.68 (±0.10),0.49 (±0.36),0.77 (±0.35),0.58 (±0.28),0.55 (±0.28),0.66 (±0.29),0.61 (±0.28)
1,gpt4v,v2u_long,0.49 (±0.29),0.58 (±0.23),0.35 (±0.18),0.10 (±0.05),0.38 (±0.28),0.57 (±0.11),0.67 (±0.10),0.60 (±0.40),0.74 (±0.41),0.62 (±0.36),0.53 (±0.32),0.60 (±0.34),0.62 (±0.35)
12,stage0,v2u_long,0.40 (±0.26),0.41 (±0.24),0.39 (±0.17),0.15 (±0.07),0.47 (±0.44),0.58 (±0.16),0.69 (±0.12),0.46 (±0.34),0.80 (±0.25),0.66 (±0.23),0.62 (±0.26),0.73 (±0.22),0.65 (±0.21)
5,cogvlm,v2u_mid,0.27 (±0.31),0.31 (±0.30),0.04 (±0.11),0.01 (±0.02),0.32 (±0.30),0.58 (±0.14),0.66 (±0.10),0.19 (±0.29),0.45 (±0.46),0.35 (±0.37),0.36 (±0.41),0.64 (±0.30),0.40 (±0.31)
7,ws,v2u_mid,0.10 (±0.14),0.11 (±0.14),0.22 (±0.25),0.03 (±0.04),0.37 (±0.38),0.59 (±0.16),0.67 (±0.11),0.13 (±0.20),0.61 (±0.32),0.56 (±0.32),0.59 (±0.35),0.73 (±0.25),0.52 (±0.23)
17,stag0ws,v2u_mid,0.16 (±0.17),0.17 (±0.18),0.11 (±0.14),0.04 (±0.04),0.34 (±0.28),0.56 (±0.16),0.67 (±0.11),0.10 (±0.18),0.45 (±0.42),0.40 (±0.38),0.33 (±0.35),0.51 (±0.38),0.36 (±0.30)
10,d2c,v2u_mid,0.45 (±0.31),0.51 (±0.28),0.31 (±0.26),0.05 (±0.05),0.35 (±0.28),0.56 (±0.14),0.70 (±0.10),0.55 (±0.35),0.84 (±0.28),0.66 (±0.24),0.62 (±0.28),0.75 (±0.23),0.69 (±0.23)


In [ ]:
keys_1=['dataset','tree_rouge_1',  'clip_sim','visual_score']
df2=df1[['dataset','model','tree_rouge_1',  'clip_sim','visual_score']].sort_values(keys_1)
#df2=df2[df2['model'] != 'stage2'].reset_index(drop=True)
df2.to_csv('/data03/starmage/projects/UICoder/outputs/dataset_track_res.csv')
df2

In [ ]:
import sys,os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

all_imgs =[]
for root,dirs,files in os.walk('/data03/starmage/projects/UICoder/outputs/eval_stage2'):
    root = Path(root)
    for file in files:
        if file.endswith('answer.png'):
            all_imgs.append([root / file, root / 'prediction.png'])


In [ ]:
idx = random.choice(range(len(all_imgs)))
fig, axs = plt.subplots(1, 2, figsize=(2*8, 2*8))   
img0 = mpimg.imread(all_imgs[idx][0]) 
img1 = mpimg.imread(all_imgs[idx][1]) 
axs[0].imshow(img0)
axs[1].imshow(img1)

In [ ]:

os.system(f'cp {str(all_imgs[idx][0])} tmp/{idx}_ref.png')
os.system(f'cp {str(all_imgs[idx][1])} tmp/{idx}_pred.png')


In [ ]:
import datasets
ds = datasets.Dataset.from_parquet("/data03/starmage/datasets/cc/final/data/02057.parquet")

In [ ]:
ds[0]['image']